In [1]:
import lightgbm as lgb
import numpy as np
import optuna.integration.lightgbm as olgb
import optuna.logging
import pandas as pd
import psutil

from lightgbm import Booster
from onnxconverter_common import FloatTensorType
from onnxmltools.convert import convert_lightgbm
from sklearn.metrics import *
from sklearn.model_selection import StratifiedKFold, train_test_split

optuna.logging.set_verbosity(optuna.logging.WARNING)

C:\Users\yuki\IdeaProjects\PhishDetector\model\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 特徴量データの読み込み
features_and_label: pd.DataFrame = pd.read_csv("./feature.csv").drop(columns=["domain"])

# featuresとlabelを分割
features = features_and_label.drop("label", axis=1)
labels = features_and_label[["label"]]

# LightGBMの形式に変換
train_data = lgb.Dataset(
    data=features,
    label=labels,
    feature_name=features.columns.tolist(),
)

In [3]:
# Stratified 5-fold CV

params = {
    "num_threads": psutil.cpu_count() - 1,  # スレッド数
    "objective": "binary",  # 二値分類
    "metric": "binary_logloss",
    "boosting_type": "gbdt",
    "verbosity": -1,  # 学習途中の情報を表示しない
    "is_unbalance": True,
    "seed": 42,
}

cv_scores_columns = ["accuracy", "TPR", "TNR", "FPR", "FNR", "F1", "AUC", "TP", "TN", "FP", "FN"]
history = np.empty((0, 11))

kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for train_idx, test_idx in kf.split(features, labels):
    x_train = features.iloc[train_idx, :]  # 学習用特徴量
    y_train = labels.iloc[train_idx, :]  # 学習用ラベル
    x_test = features.iloc[test_idx, :]  # テスト用特徴量
    y_test = labels.iloc[test_idx, :]  # テスト用ラベル

    dtrain = lgb.Dataset(x_train, label=y_train, feature_name=features.columns.tolist())
    dtest = lgb.Dataset(x_test, label=y_test, feature_name=features.columns.tolist())

    # 訓練
    model = olgb.train(
        params=params,
        train_set=dtrain,
        valid_sets=[dtrain, dtest],
        callbacks=[lgb.early_stopping(100, verbose=False)],
        num_boost_round=10000,
    )

    # 予測
    label_pred_prob = model.predict(x_test)
    label_pred = (label_pred_prob > 0.5).astype(int)

    # 評価指標の計算
    tn, fp, fn, tp = confusion_matrix(y_test, label_pred).flatten()
    acc = accuracy_score(y_test, label_pred)
    tpr = tp / (tp + fn)  # recall
    tnr = tn / (tn + fp)  # specificity
    fpr = fp / (tn + fp)
    fnr = fn / (tp + fn)
    f1 = f1_score(y_test, label_pred)
    auc = roc_auc_score(y_test, label_pred_prob)

    # 評価結果の格納
    history = np.vstack([history, [acc, tpr, tnr, fpr, fnr, f1, auc, tp, tn, fp, fn]])

min_child_samples, val_score: 0.014361: 100%|##########| 5/5 [00:01<00:00,  4.54it/s]


In [4]:
mean = np.mean(history, axis=0)
history = np.vstack([history, mean])

df_cv_scores = pd.DataFrame(history, columns=cv_scores_columns)
df_cv_scores

,accuracy,TPR,TNR,FPR,FNR,F1,AUC,TP,TN,FP,FN
0,0.987879,0.996403,0.942308,0.057692,0.003597,0.992832,0.999723,277.0,49.0,3.0,1.0
1,0.987879,0.992806,0.961538,0.038462,0.007194,0.992806,0.997233,276.0,50.0,2.0,2.0
2,0.990909,0.996403,0.961538,0.038462,0.003597,0.994614,0.994881,277.0,50.0,2.0,1.0
3,0.984802,0.989209,0.960784,0.039216,0.010791,0.990991,0.996403,275.0,49.0,2.0,3.0
4,0.996960,0.996403,1.000000,0.000000,0.003597,0.998198,0.999718,277.0,51.0,0.0,1.0
5,0.989686,0.994245,0.965234,0.034766,0.005755,0.993888,0.997592,276.4,49.8,1.8,1.6


In [5]:
# 訓練
x_train, x_valid, y_train, y_valid = train_test_split(features, labels, test_size=0.2, stratify=labels)

dtrain = lgb.Dataset(x_train, label=y_train, feature_name=features.columns.tolist())
dvalid = lgb.Dataset(x_valid, label=y_valid, feature_name=features.columns.tolist())

# 訓練
model = olgb.train(
    params=params,
    train_set=dtrain,
    valid_sets=[dtrain, dvalid],
    callbacks=[lgb.early_stopping(100, verbose=False)],
    num_boost_round=10000,
)

importance = pd.DataFrame(
    model.feature_importance(),
    index=features.columns.tolist(),
    columns=['importance']
).sort_values(by='importance', ascending=False)

importance

min_child_samples, val_score: 0.037420: 100%|##########| 5/5 [00:00<00:00,  8.21it/s]


,importance
tag_count_in_head_tag,355
script_tag_count,306
external_link_percentage,251
no_domain_in_internal_link,192
same_page_link_count,89
iframe_tag_count,81
google_analytics,32
invalid_kiyaku,26
copied,5
no_title,1


In [12]:
# ONNXへの変換
initial_type = [("float_input", FloatTensorType(None, 11))]
onx = convert_lightgbm(model=model, initial_types=initial_type, zipmap=False)
with open("lightgbm.onnx", "wb") as f:
    f.write(onx.SerializeToString())

The maximum opset needed by this model is only 9.
